# Exercises
In this notebook, we will explore the capabilities of transformer-based models for natural language processing (NLP) using the Hugging Face `transformers` library. We will use the `sentence-transformers` package to extract features from text data and the `transformers` library to perform text generation tasks.

By the end of this exercise, you will have learned how to:
- Extract features from text data using transformer-based models
- Generate text using transformer-based models
- Access the largest and latest open models through the Hugging Face API


## Using Notebook Environments 
1. To run a cell, press `shift + enter`. The notebook will execute the code in the cell and move to the next cell. If the cell contains a markdown cell (text only), it will render the markdown and move to the next cell.
2. Since cells can be executed in any order and variables can be over-written, you may at some point feel that you have lost track of the state of your notebook. If this is the case, you can always restart the kernel by clicking Runtime in the menu bar (if you're using Colab) and selecting `Restart runtime`. This will clear all variables and outputs.
3. The final variable in a cell will be printed on the screen. If you want to print multiple variables, use the `print()` function as usual.

Notebook environments support code cells and markdown (text) cells. For the purposes of this workshop, markdown cells are used to provide high-level explanations of the code. More specific details are provided in the code cells themselves in the form of comments (lines beginning with `#`)

## Environment Setup

In [ ]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environment
    
    # Installing requisite packages
    !pip install --upgrade transformers sentence-transformers pacmap accelerate &> /dev/null

    # Change working directory to day_1
    %cd /content/drive/MyDrive/LLM4BeSci_Bern2024

We begin by loading the requisite packages. For those coming from R, packages in Python are sometimes given shorter names for use in the code via the `import <name> as <nickname>` syntax (e.g. `import pandas as pd`). These are usually standardized nicknames. We here make use three packages:

1. `pandas`: A very popular package for reading and manipulating data in python.
2. `sentence_transformers`: A package for extracting features from text data using transformer-based models.
3. `transformers`: A HF package for loading and manipulating transformer-based models.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Feature Extraction: Relating Personality Measures

In this analysis--inspired by  [Wulff & Mata, 2023](https://osf.io/preprints/psyarxiv/9h7aw)--we will use an LLM to extract features from personality items. In this case, features are *numerical* representations of the meaning (and syntax) of text. We will extract features using the `sentence-transformers` package.
The code will use the extracted features to compute the similarity between personality items. It then evaluates how well these similarities predict the observed similarities between items. Finally, it visualises the item similarity space in two dimensions. 

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pacmap import PaCMAP

The code begins by loading the personality items into a `pandas.DataFrame` with three columns:

1. `factor`: The (high-level) personality factor to which the item belongs.
2. `construct`: The (mid-level) personality construct to which the item belongs.
3. `item`: The text of the personality item used to measure the construct.

Run the cell below.

In [ ]:
# Loading personality data
personality = pd.read_csv('items.csv') 
personality

### Extracting Features from Personality Items
The code makes use of the `all-MiniLM-L6-v2` model, which is a small and efficient embedding model, to extract features from the sentences. The model will encode the sentences into 384-dimensional vector representations. The cell will then print the features as a pandas dataframe for easy viewing. 

In [ ]:
# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract features
item_features = model.encode(personality['item'])

# Print the features as a pandas dataframe
item_features = pd.DataFrame(item_features, index=personality['item'])
item_features

### Computing Similarity between Personality Items
Now that we have extracted features for each personality item, we can compute the similarity between items. We use the `sklearn`'s `cosine similarity` function, which measures the cosine of the angle between two vectors. The closer the cosine similarity is to 1, the more similar the two items are. We compute the similarity between all pairs of items and store the results in a similarity matrix.

Run the cell below.

In [ ]:
# Compute cosine similarity between features
predicted_sims = cosine_similarity(item_features)
predicted_sims = pd.DataFrame(predicted_sims, index=personality['item'], columns=personality['item'])
predicted_sims

As you can see, the similarity matrix is symmetric, with the diagonal containing 1s (since the similarity of an item with itself is 1). Furthermore, items that you would expect to be more related (e.g. "Turn plans into actions" and "Plunge into tasks with all my heart." are indeed more similar. Conversely, less related items (e.g. "Am calm in tense situations" and "Demand quality") show lower cosine similarities.

 ### Comparing to observed correlations between items
This section compares how well the predicted similarities align with the *observed* similarities between items: that is, the correlations between the participant responses to the items. It first loads the observed correlations into a `pandas.DataFrame`:

In [ ]:
# Load observed correlations
observed_sims = pd.read_csv('item_corrs.csv')
observed_sims

Next, the code pivots `observed_sims` to create a correlation matrix with the same structure as `predicted_sims` so that they can be easily compared.

In [ ]:
# Pivoting to a correlation matrix for easy comparison with predicted correlations
observed_sims = observed_sims.pivot(index='text_i', columns='text_j', values='cor')
observed_sims

The predicted and observed similarities are then aligned to ensure that the items are in the same order. The code then flattens the lower triangle of the matrices into vectors to compute the correlation between the predicted and observed similarities.

In [ ]:
# Aligning observed and predicted similarities
predicted_sims, observed_sims = predicted_sims.align(observed_sims)

def lower_triangle_flat(df):
    """Takes the lower triangle of a dataframe and flattens it into a vector"""
    rows, cols = np.triu_indices(len(df), k=1)  # k=1 to exclude the diagonal (self-similarities)
    return pd.Series(df.values[rows, cols])

# Flatten the lower triangle of the observed and predicted similarities into vectors
predicted_sims_flat, observed_sims_flat = lower_triangle_flat(predicted_sims), lower_triangle_flat(observed_sims)

# Correlation between predicted and observed
print(f'r: {predicted_sims_flat.corr(observed_sims_flat).round(2)}')
print(f'r of absolute values: {predicted_sims_flat.abs().corr(observed_sims_flat.abs()).round(2)}')

The correlation between the predicted and observed similarities is 0.18. If we take the absolute values of the similarities, the correlation increases to 0.23. Since we are not interested in which way round (in terms of polarity) the personality item scale was rated, we focus on the absolute values. This suggests that the extracted features capture some of the variance in the observed similarities between items. Whilst this suggests that the extracted features may not be capturing everything we want to know about the items, alternative explanations exist. Can you think of any?

### Visualizing the Item Similarities
We can also visualize `predicted_sims` in two dimensions using PaCMAP. PaCMAP is a dimensionality reduction technique that preserves the pairwise distances between points. The code fits the PaCMAP model to the extracted features and transform them into two dimensions, saving the results in a `pandas.DataFrame`. 

In [ ]:
# Initialize MDS model
pac = PaCMAP(n_components=2, random_state=42)

# Fit and transform the features
pac_features = pac.fit_transform(item_features)

# Convert features to DataFrame
pac_features = pd.DataFrame(pac_features, columns=['x', 'y'])
pac_features

Next, the code adds the personality factors and items as columns to `pac_features` to see how items cluster based on their similarity. 

In [ ]:
# Adding personality factors to MDS features
pac_features['factor'] = personality['factor']
pac_features['item'] = personality['item']
pac_features

The code next plots the MDS features, with each point representing a personality item. The points are colored by factor, allowing us to see how items cluster based on their similarity.

In [ ]:
# Plot pac features
sns.scatterplot(data=pac_features, x='x', y='y', hue='factor', s=100)
sns.despine(offset=10)

As illustrated, the items somewhat cluster according to their factor, again suggesting that the extracted features have captured some meaningful information about the items.

## Text Generation: Phi-3 Takes the Berlin Numeracy Test
In this analysis, we will explore the usage of causal LLMs as synthetic participants in a psychological experiment. We will again use Phi-3, this time to solve the [Berlin Numeracy Test](https://doi.org/10.1017/S1930297500001819). This is a widely used test to measure an individual's ability to understand and apply statistical concepts. 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import textwrap

In [ ]:
q1 = """
Imagine we are throwing a five-sided die 50 times. On average, out of these 50 throws how many times would this five-sided die show an odd number (1, 3 or 5)?
"""

q2 = """
Out of 1,000 people in a small town 500 are members of a choir. Out of these 500 members in the choir 100 are men. Out of the 500 inhabitants that are not in the choir 300 are men. What is the probability that a randomly drawn man is a member of the choir? (please indicate the probability in percent).
"""

q3 = """
Imagine we are throwing a loaded die (6 sides). The probability that the die shows a 6 is twice as high as the probability of each of the other numbers. On average, out of these 70 throws, how many times would the die show the number 6?
"""

q4 = """
In a forest 20% of mushrooms are red, 50% brown and 30% white. A red mushroom is poisonous with a probability of 20%. A mushroom that is not red is poisonous with a probability of 5%. What is the probability that a poisonous mushroom in the forest is red?
"""

It next loads Phi-3. Since we will be using Phi-3 to generate text, Phi-3 is loaded using the `transformers` `AutoModelForCausalLM` class. We also load the `AutoTokenizer` class to tokenize the input text.

In [ ]:
torch.random.manual_seed(42) # Set seed for reproducibility

# Load Phi-3
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda", # Use GPU
    torch_dtype=torch.float16,  # Use float16 for faster inference
    trust_remote_code=True, 
    attn_implementation='eager' # Faster inference on T4 GPUs
)

# Load tokenizer`
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

The code next initialises the familiar `'text-generation'` `pipeline` from day 3. 

In `generation_args` (which is later fed into the `pipeline` during generation), the code sets `"max_new_tokens": 100` to limit the length of the model output and `"do_sample": False` to use of *greedy decoding*, which means that the model will always choose the most likely token at each step. 

In [ ]:
# Define text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# Define generation arguments
generation_args = {
    "max_new_tokens": 100,  # Maximum number of tokens to generate
    "return_full_text": False, # Return only the generated text
    "do_sample": False, # Use greedy decoding (incompatible with temperature>0)
    # "temperature": 0.0  # Uncomment this line to set the temperature parameter
}

# Loop through questions and generate responses
for i, question in enumerate([q1, q2, q3, q4]):
    print('-------------------------')   
    
    # Define prompt
    prompt = [
        {"role": "system", "content": "You are a thoughtful participant in a psychology experiment. Please answer the following question making your answer as brief as possible (less than 50 words)."},
        {"role": "user", "content": question}
    ]  # Define prompt with JSON structure
    
    # Generate response and access generated text at index 0 and key 'generated_text'
    response = pipe(prompt, **generation_args)[0]['generated_text']
    
    # Format question and response for printing
    question = '\n'.join(textwrap.wrap(question, 100))
    response = '\n'.join(textwrap.wrap(response, 100))
    print(f"Question {i+1}: {question} \n\nAnswer: {response}\n")

### Using the Hugging Face API
This section demonstrates how to use the Hugging Face API to do the same as the above numeracy test example. The main benefit of the API is that it allows us to run the latest, largest models without having the specialised hardware needed to run them (since the models are run on the cloud). We will use the `meta-llama/Meta-Llama-3-70B-Instruct` model. 

In order to use the Hugging Face API, you will need to follow the following steps:

1.  Make sure you have a Hugging Face account (https://huggingface.co/join)
2. Go to the [Llama-3 model page](https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct) and fill in the 'META LLAMA 3 COMMUNITY LICENSE AGREEMENT' form at the top of the page in order to get access to the model (this can take up to an hour). 
3. Once you have been granted access, you can navigate to [in your Hugging Face profile settings](https://huggingface.co/settings/tokens) to get the model's API access token. This token should provide access to all models in the Llama family. 
4. If you wish to run the largest (70B parameter) version of Llama-3, you will need to have a Hugging Face PRO subscription (currently $9/month). You can also run the smaller versions (as in this notebook) for free.

The Hugging Face API code begins importing the `InferenceClient` class from the `huggingface_hub` library. The `InferenceClient` provides a high-level API to interact with models hosted on the Hugging Face Hub. 

In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
# Initialize client
api_key = '<your access token here>' 
client = InferenceClient(token=api_key)

for i, question in enumerate([q1, q2, q3, q4]):
    print('-------------------------')   
    
    prompt = [
        {"role": "system", "content": "You are a thoughtful participant in a psychology experiment. Please answer the following question making your answer as brief as possible (less than 50 words)."},
        {"role": "user", "content": question}
    ]  # Define prompt with JSON structure
    
    response = client.chat_completion(
        messages=prompt,
        model="meta-llama/Meta-Llama-3-70B-Instruct",
        max_tokens=100,
        temperature=0.0
    ).choices[0].content
    
    # Format question and response for printing
    question = '\n'.join(textwrap.wrap(question, 100))
    response = '\n'.join(textwrap.wrap(response, 100))
    print(f"Question {i+1}: {question} \n\nAnswer: {response}\n")